In [1]:
# Imports
import math
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed,
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

2026-01-18 16:19:32.242681: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768753172.468782      23 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768753172.539337      23 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768753173.082882      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768753173.082917      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768753173.082920      23 computation_placer.cc:177] computation placer alr

In [2]:
# Config/Path vars

SEED = 42

root_path = Path("/kaggle/input/crowdflower-weather-twitter")
sample_sub_path = root_path / "sampleSubmission.csv"
train_csv_path = root_path / "train.csv"
test_csv_path = root_path / "test.csv"
var_names_txt_path = root_path / "variableNames.txt"
model_id = "vinai/bertweet-base"

In [3]:
# Utility functions/configs

rng_np = np.random.default_rng(SEED)

In [4]:
train = pd.read_csv(train_csv_path)
test = pd.read_csv(test_csv_path)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77946 entries, 0 to 77945
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        77946 non-null  int64  
 1   tweet     77946 non-null  object 
 2   state     77946 non-null  object 
 3   location  66994 non-null  object 
 4   s1        77946 non-null  float64
 5   s2        77946 non-null  float64
 6   s3        77946 non-null  float64
 7   s4        77946 non-null  float64
 8   s5        77946 non-null  float64
 9   w1        77946 non-null  float64
 10  w2        77946 non-null  float64
 11  w3        77946 non-null  float64
 12  w4        77946 non-null  float64
 13  k1        77946 non-null  float64
 14  k2        77946 non-null  float64
 15  k3        77946 non-null  float64
 16  k4        77946 non-null  float64
 17  k5        77946 non-null  float64
 18  k6        77946 non-null  float64
 19  k7        77946 non-null  float64
 20  k8        77946 non-null  fl

In [6]:
for i in range(2):
    n = rng_np.integers(low=0, high=77946) 
    sample = train['tweet'][n]
    state = train.get('state')[n]
    location = train.get('location')[n]
    print('-'*80)
    print(f'Index: {n};\nSample:\n{sample};\nState: {state};\nLocation: {location}')

--------------------------------------------------------------------------------
Index: 6956;
Sample:
One gallon of milk.  Seven hours left in the back of the car.  Ninety degrees. You do the math.;
State: wyoming;
Location: Midwest
--------------------------------------------------------------------------------
Index: 60326;
Sample:
RT @mention: Pro-am draws stars, raises funds for storm-battered Alabama:  Alabama football coach Nick Saban spends much of hi... ...;
State: alabama;
Location: Tuscaloosa, AL


In [7]:
# Make dataset
non_label = {"id", "tweet", "state", "location"}
candidates = [c for c in train.columns if c not in non_label]
label_cols = [c for c in candidates if (c.startswith("s") or c.startswith("w") or c.startswith("k"))]
# Sort in the  Kaggle order (s*, then w*, then k*) by prefix then numeric suffix.
def key_fn(c):
    prefix = c[0]
    try:
        num = int(c[1:])
    except Exception:
        num = 10**9
    prefix_order = {"s": 0, "w": 1, "k": 2}.get(prefix, 9)
    return (prefix_order, num, c)
    
label_cols = sorted(label_cols, key=key_fn)

train_df = train.copy()
train_df['tweet'] = train_df['tweet'].astype(str)
tr_df, va_df = train_test_split(train_df, test_size=0.2, random_state=SEED, shuffle=True)

ds_train = Dataset.from_pandas(tr_df.reset_index(drop=True))
ds_val = Dataset.from_pandas(va_df.reset_index(drop=True))

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

def tokenize_and_format(ds: Dataset, tokenizer, label_cols, text_col="tweet", is_train=True):
    def tok(batch):
        return tokenizer(
            batch[text_col],
            truncation=True
        )

    ds = ds.map(tok, batched=True)

    if is_train:
        def add_labels(batch):
            labels = []
            for i in range(len(batch[text_col])):
                labels.append([float(batch[c][i]) for c in label_cols])
            batch["labels"] = labels
            return batch

        ds = ds.map(add_labels, batched=True)
        ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    else:
        ds.set_format(type="torch", columns=["input_ids", "attention_mask"])
    return ds


ds_train = tokenize_and_format(ds_train, tokenizer, label_cols, is_train=True)
ds_val = tokenize_and_format(ds_val, tokenizer, label_cols, is_train=True)

config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/62356 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/62356 [00:00<?, ? examples/s]

Map:   0%|          | 0/15590 [00:00<?, ? examples/s]

Map:   0%|          | 0/15590 [00:00<?, ? examples/s]

In [8]:
num_labels = len(label_cols)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=num_labels, problem_type="multi_label_classification")

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
def compute_metrics_rmse(eval_pred):
    """
    Kaggle uses RMSE. We'll compute RMSE over all outputs by flattening:
    RMSE = sqrt(mean((y - yhat)^2)).
    """
    logits, labels = eval_pred
    # For multi-label, model outputs logits. Convert to probabilities via sigmoid.
    probs = 1.0 / (1.0 + np.exp(-logits))
    rmse = math.sqrt(mean_squared_error(labels.reshape(-1), probs.reshape(-1)))
    return {"rmse": rmse}

In [10]:
# training_args = TrainingArguments(
#     output_dir="out",
#     learning_rate=2e-5,
#     per_device_train_batch_size=100,
#     per_device_eval_batch_size=16,
#     gradient_accumulation_steps=2,
#     num_train_epochs=10,
#     weight_decay=0.01,
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     metric_for_best_model="rmse",
#     greater_is_better=False,
#     logging_steps=200,
#     fp16=True,
#     report_to="none",
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=ds_train,
#     eval_dataset=ds_val,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics_rmse,
# )

# trainer.train()
# eval_metrics = trainer.evaluate()
# print("Final eval:", eval_metrics)

In [11]:
test_df = test.copy()
test_df["tweet"] = test_df["tweet"].astype(str)

ds_test = Dataset.from_pandas(test_df.reset_index(drop=True))
ds_test = tokenize_and_format(ds_test, tokenizer, label_cols, is_train=False)

ckpt_path = "/kaggle/input/berttweet-partly-sunny-with-a-chance-of-hashtags/transformers/default/1/checkpoint"
config = AutoConfig.from_pretrained(ckpt_path)
trained_model = AutoModelForSequenceClassification.from_pretrained(ckpt_path, config=config)

pred_args = TrainingArguments(
    output_dir="/kaggle/working/pred_tmp",
    per_device_eval_batch_size=64,
    dataloader_num_workers=0,   
    report_to="none"
)

trainer = Trainer(model=trained_model, tokenizer = tokenizer, args=pred_args)

pred = trainer.predict(ds_test)
probs = torch.sigmoid(torch.from_numpy(pred.predictions)).numpy()

sub = pd.DataFrame(probs, columns=label_cols)
sub.insert(0, "id", test_df["id"].values)
sub.to_csv("/kaggle/working/submission.csv", index=False)
print("All done!")

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Map:   0%|          | 0/42157 [00:00<?, ? examples/s]

/tmp/ipykernel_23/3034573692.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=trained_model, tokenizer = tokenizer, args=pred_args)


All done!
